In [ ]:
! pip install lightning-flash
! pip install 'lightning-flash[audio,text]'

In [22]:
! pip install jiwer

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for jiwer from https://files.pythonhosted.org/packages/23/a3/92c29a5e422acd87e3b4f2e6dc0ce877070cc9b2f81d30fe84122032338a/jiwer-3.0.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.8 MB/s eta 0:00:00


In [34]:
! pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.5/553.5 kB 24.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for hypothesis<7,>=6.14 from https://files.pythonhosted.org/packages/f7/af/c94ff33c5ba640d17e13cdfd17fc90bc245df1f285a6d418f1fc7d9610d8/hypothesis-6.82.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.9/414.9 kB 31.5 MB/s eta 0:00:00
  Created wheel for kenlm: filename=kenlm-0.0.0-cp38-cp38-linux_x86_64.whl size=3214997 sha256=f4db6c3f2007c499b580bccad2fca7ccba83d116fc0952bd48bdfd8076842da6
  Stored in directory: /tmp/pip-ephem-wheel-cache-jv20egqq/wheels/ff/08/4e/a3ddc0e786e0f3c1fcd2e7a82c4324c02fc3ae2638471406d2
Successfully built kenlm


In [4]:
import flash
import torch
from flash.audio import SpeechRecognition, SpeechRecognitionData

In [ ]:

# 2. Build the task
model = SpeechRecognition(backbone="facebook/wav2vec2-large-960h-lv60-self")


In [14]:
wav2vec2_trainer = flash.Trainer(accumulate_grad_batches=1,
                                              precision=16,
                                              max_epochs=1,
                                              gpus=1,
                                              num_nodes = 1
                                              )

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [213]:
wav2vec2_model = SpeechRecognition.load_from_checkpoint("/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt")

Using 'facebook/wav2vec2-large-robust-ft-libri-960h' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Using 'facebook/wav2vec2-large-robust-ft-libri-960h' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).


In [388]:
#wav2vec2_model = SpeechRecognition.load_from_checkpoint("/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt")
wav2vec2_model = SpeechRecognition.load_from_checkpoint("/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt")

Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [402]:
import pandas as pd
test_data = pd.read_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_song_metadata_en_demucs_cleaned.csv")

In [403]:
test_data.head(2)

,Folder,File,consolidated_file_path,Length,file_name,transcription,language,transcription_capitalized
0,9fdeb5c63854498da6defbca63f128be,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.509002,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,just a tryin' to survive,en,JUST A TRYIN' TO SURVIVE
1,d2ad6272e46440cd9239e6186de5e74a,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.062948,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,don't go on me,en,DON'T GO ON ME


In [404]:
test_datamodule = SpeechRecognitionData.from_files(
    predict_files=list(test_data["consolidated_file_path"]), 
    batch_size=1)

finetuned_predictions = wav2vec2_trainer.predict(wav2vec2_model, 
                                                datamodule=test_datamodule)

base_pred = wav2vec2_trainer.predict(model, 
                                                datamodule=test_datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [405]:
test_data.head(5)

,Folder,File,consolidated_file_path,Length,file_name,transcription,language,transcription_capitalized
0,9fdeb5c63854498da6defbca63f128be,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.509002,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,just a tryin' to survive,en,JUST A TRYIN' TO SURVIVE
1,d2ad6272e46440cd9239e6186de5e74a,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,3.062948,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,don't go on me,en,DON'T GO ON ME
2,a76d6dfe717b460ea33b7e6e86df183d,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,5.003084,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,i don't know how i feel,en,I DON'T KNOW HOW I FEEL
3,1cede001e02c41bea6a63569465e7b03,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.757007,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,i would love to love you lover,en,I WOULD LOVE TO LOVE YOU LOVER
4,9a7cc26706da4e0e92c61608c32d9580,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,7.486757,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,i am the greatest man that ever lived,en,I AM THE GREATEST MAN THAT EVER LIVED


In [406]:
base_pred_transformed = []
for predictions in base_pred:
    base_pred_transformed.append(predictions[0])


In [407]:
finetuned_pred_transformed = []
for predictions in finetuned_predictions:
    finetuned_pred_transformed.append(predictions[0])


In [408]:
test_data["base_pred"] = base_pred_transformed
test_data["finetuned_predictions"] = finetuned_pred_transformed

In [409]:
from jiwer import wer

reference = list(test_data["transcription_capitalized"])
hypothesis = base_pred_transformed

error = wer(reference, hypothesis)
error

0.7814730100640439

In [410]:
from jiwer import wer

reference = list(test_data["transcription_capitalized"])
hypothesis = finetuned_pred_transformed

error = wer(reference, hypothesis)
error

0.7814730100640439

In [411]:
test_data.loc[50:55,:]

,Folder,File,consolidated_file_path,Length,file_name,transcription,language,transcription_capitalized,base_pred,finetuned_predictions
50,3388279af3904205916cdc0ae818c398,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.022449,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,i'm the one who takes you there,en,I'M THE ONE WHO TAKES YOU THERE,I'M NO GOING TO TAKE YOU THERE,I'M NO GOING TO TAKE YOU THERE
51,23ff388c931d4fe5af9e87eea597defd,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,1.607506,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,cuz you're the baddest little thing,en,CUZ YOU'RE THE BADDEST LITTLE THING,I'S AL THE BAES SIN THING,I'S AL THE BAES SIN THING
52,d89513df28a946c193d564b7b9c4007d,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.100000,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,have no fear of death,en,HAVE NO FEAR OF DEATH,O YO LI O,O YO LI O
53,a88718deaae24732886fdc197dc845a8,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,4.588254,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,unending answer to all your fears,en,UNENDING ANSWER TO ALL YOUR FEARS,I'M LENTING ANSER TO ALL YOUR PERS,I'M LENTING ANSER TO ALL YOUR PERS
54,711e8893e10e498f9086c0d7c6c1947d,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,2.175011,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,you're gonna be just fine,en,YOU'RE GONNA BE JUST FINE,WE GON TO MAKE TIS FAT,WE GON TO MAKE TIS FAT
55,0feb990352bf494f95ae7e3417cf8559,vocals.wav,/home/users/gmenon/workspace/songsLyricsGenera...,4.000000,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,has darkness taken over me,en,HAS DARKNESS TAKEN OVER ME,AND NO THE SAV E OI,AND NO THE SAV E OI


In [400]:
# # load pretrained model
# from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
# asr_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# asr_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [401]:
# from transformers import Wav2Vec2ProcessorWithLM

# processor = Wav2Vec2ProcessorWithLM.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm")

In [77]:
import pyctcdecode

In [5]:
vocab = {"<pad>": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31}

In [6]:
labels = list(vocab.keys())

In [7]:
import torchaudio
waveform, sample_rate = torchaudio.load('/home/users/gmenon/workspace/songsLyricsGenerator/data/DALI_v1.0/audio/wav_clips/separated/htdemucs/a88718deaae24732886fdc197dc845a8/vocals.wav')
waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)

In [8]:
waveform.squeeze().shape
#model = SpeechRecognition.load_from_checkpoint("/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt")


torch.Size([2, 73413])

In [10]:
import numpy as np
import librosa
waveform = librosa.to_mono(np.array(waveform))
waveform

array([ 0.00172427,  0.00930336,  0.00756506, ..., -0.11694409,
       -0.07854679, -0.01719929], dtype=float32)

In [11]:
waveform.shape

(73413,)

In [12]:
# from torchaudio.models.decoder import ctc_decoder, download_pretrained_files
# files = download_pretrained_files("librispeech-4-gram")
# decoder = ctc_decoder(
#    lexicon=files.lexicon,
#    tokens=files.tokens,
#    lm=files.lm,
#    nbest=1
# )

In [13]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
hf_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = SpeechRecognition.load_from_checkpoint("/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt")


 

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using 'facebook/wav2vec2-large-960h-lv60-self' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# from datasets import load_dataset

# dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
# dataset

In [15]:
# dataset["audio"][0]["array"].shape, waveform.T.squeeze().shape

In [16]:
# inputs = processor(dataset["audio"][0]["array"], sampling_rate=16_000, return_tensors="pt")
# inputs["input_values"].shape

In [17]:
inputs = processor(waveform.T.squeeze(), sampling_rate=16_000, return_tensors="pt")
inputs["input_values"].shape

torch.Size([1, 73413])

In [18]:
# inputs = {k: v.to("cuda") for k,v in inputs.items()}
# inputs

In [19]:
waveform.shape

(73413,)

In [20]:
inputs = processor(waveform, sampling_rate=16_000, return_tensors="pt")
inputs["input_values"].shape

torch.Size([1, 73413])

In [21]:
import torch

with torch.no_grad():
  logits = model.model(**inputs).logits

In [22]:
logits.shape

torch.Size([1, 229, 32])

In [23]:
# from pyctcdecode import build_ctcdecoder
vocab = {"<pad>": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31}
vocab_list = list(vocab.keys())
# decoder = build_ctcdecoder(vocab_list)
# decoder.decode(logits.squeeze())

In [24]:
def _greedy_decode(logits,labels):
    """Decode argmax of logits and squash in CTC fashion."""
    label_dict = {n: c for n, c in enumerate(labels)}
    prev_c = None
    out = []
    for x in logits.argmax(axis=1):
        for n in x:
            c = label_dict.get(n.item(), "")  # if not in labels, then assume it's ctc blank char
            if c != prev_c:
                out.append(c)
            prev_c = c
    return "".join(out)

In [25]:
_greedy_decode(logits,vocab_list)

'KNUUD'

In [26]:
from torchaudio.models.decoder import download_pretrained_files

files = download_pretrained_files("librispeech-4-gram")

print(files)

PretrainedFiles(lexicon='/home/users/gmenon/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lexicon.txt', tokens='/home/users/gmenon/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/tokens.txt', lm='/home/users/gmenon/.cache/torch/hub/torchaudio/decoder-assets/librispeech-4-gram/lm.bin')


In [27]:
logits.squeeze()[0]

tensor([  9.9043, -22.8711, -22.5385, -22.3990,  -7.7792,  -0.8127,   1.0707,
          0.9914,  -0.8369,  -0.6631,   2.3691,  -1.1760,   1.9395,  -3.5349,
         -0.7451,  -1.7523,  -1.9744,  -2.6891,  -0.5868,  -2.0085,  -2.5887,
         -3.4665,  -0.8007,  -1.5650,  -1.4590,  -3.6714,  -1.9487,  -2.5568,
         -4.8734,  -3.8054,  -6.8474,  -2.7006])

In [28]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(vocab_list)
print(decoder.decode(logits[0].cpu().detach().numpy()))
#print(decoder.decode_beams(logits[0].cpu().detach().numpy()))[]

Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


I'M LENTING ANSER TO ALL YOUR PERS


In [29]:
#! pwd (/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/src/notebooks)

# !pip install https://github.com/kpu/kenlm/archive/master.zip

In [31]:
import kenlm
kenlm_model = kenlm.Model('/home/users/gmenon/workspace/songsLyricsGenerator/src/kenlm/4gram_big.arpa')


Loading the LM will be faster if you build a binary file.
Reading /home/users/gmenon/workspace/songsLyricsGenerator/src/kenlm/4gram_big.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [34]:
decoder = build_ctcdecoder(
    asr_model.decoder.vocabulary,
    kenlm_model,
    unigram_list,
)
decoder.decode(logits)

TypeError: Cannot convert <class 'kenlm.Model'> to string

In [5]:
# ! git clone --recursive https://github.com/parlance/ctcdecode.git
# ! cd ctcdecode && pip install .

fatal: destination path 'ctcdecode' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Processing /mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/src/notebooks/ctcdecode
  Preparing metadata (setup.py) ... done
  Created wheel for ctcdecode: filename=ctcdecode-1.0.3-cp38-cp38-linux_x86_64.whl size=19313592 sha256=400a09d0d9f8029080bfafd416a31944a79dc6a39f001c2b7dc6ae6c618da92a
  Stored in directory: /tmp/pip-ephem-wheel-cache-gr_knuf_/wheels/51/ba/8e/ad9755178fe9d27230218c9fd8c6dd3934ef9606f61e3b6598
Successfully built ctcdecode


In [566]:
# print(model.score("i'm lenting answe to all your pers", bos = True, eos = True))

-21.547266006469727


In [35]:
import ctcdecode

beam_decoder = CTCBeamDecoder(vocab_list, 
                                    lm_path='/home/users/gmenon/workspace/songsLyricsGenerator/src/kenlm/4gram_big.arpa',
                                     alpha=0.5,
                                     beta=1,
                                     cutoff_top_n=40,
                                     cutoff_prob=1.0,
                                     beam_width=100, 
                                     num_processes=16,
                                     blank_index=0)

Exception ignored in: <function CTCBeamDecoder.__del__ at 0x7fff24b6eb80>
Traceback (most recent call last):
  File "/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/ctcdecode/__init__.py", line 139, in __del__
    if self._scorer is not None:
AttributeError: 'CTCBeamDecoder' object has no attribute '_scorer'


TypeError: __init__() got an unexpected keyword argument 'lm_path'

In [36]:
from ctcdecode import CTCBeamDecoder

In [39]:
from ctcdecode import CTCBeamDecoder

decoder = CTCBeamDecoder(
    labels,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=100,
    num_processes=4,
    blank_id=0,
    log_probs_input=False
)
beam_results, beam_scores, timesteps, out_lens = decoder.decode(logits)

In [59]:
label_dict = {n: c for n, c in enumerate(vocab_list)}
print(beam_results.shape)
for n in beam_results.squeeze():
    print("hello")
    for x in n:
        c = label_dict.get(n.item(), "")
        print(c)
    

torch.Size([1, 100, 229])
hello


RuntimeError: a Tensor with 229 elements cannot be converted to Scalar

In [567]:
vocab_list

['<pad>',
 '<s>',
 '</s>',
 '<unk>',
 '|',
 'E',
 'T',
 'A',
 'O',
 'N',
 'I',
 'H',
 'S',
 'R',
 'D',
 'L',
 'U',
 'M',
 'W',
 'C',
 'F',
 'G',
 'Y',
 'P',
 'B',
 'V',
 'K',
 "'",
 'X',
 'J',
 'Q',
 'Z']

In [ ]:
decoder = CTCBeamDecoder(
    labels,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=100,
    num_processes=4,
    blank_id=0,
    log_probs_input=False
)

In [ ]:
beam_search_result = beam_search_decoder(logits)
beam_search_result

In [ ]:
beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
beam_search_transcript

In [ ]:
timesteps = beam_search_result[0][0].timesteps
predicted_tokens = beam_search_decoder.idxs_to_tokens(beam_search_result[0][0].tokens)

print(predicted_tokens, len(predicted_tokens))
print(timesteps, timesteps.shape[0])

In [523]:
# from pyctcdecode import build_ctcdecoder
# vocab = {"<pad>": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31}
# vocab_list = list(vocab.keys())
# decoder = build_ctcdecoder(vocab_list)
# decoder.decode(logits)

In [477]:
beam_search_wer = torchaudio.functional.edit_distance(actual_transcript, beam_search_result[0][0].words) / len(
    actual_transcript
)

print(f"Transcript: {beam_search_transcript}")
print(f"WER: {beam_search_wer}")

NameError: name 'actual_transcript' is not defined